<a href="https://colab.research.google.com/github/Kurt9806/FEM.BEAMUPC/blob/master/Buckling_IFSDT_Micropolar_Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
from scipy.linalg import eig
from matplotlib import pyplot as plt
from scipy.io import loadmat
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 0-Cantilever-tip load_Hassanpour_Fig4_paper.mat to 0-Cantilever-tip load_Hassanpour_Fig4_paper.mat


In [ ]:
Dict = loadmat('0-Cantilever-tip load_Hassanpour_Fig4_paper.mat')
b = float(Dict['b'][0][0])
g = float(Dict['g'][0][0])
ka = float(Dict['kah'][0][0])
u = float(Dict['u'][0][0])
E = float(Dict['E'][0][0])
Hh = float(Dict['Hh'][0][0])
Bb = float(Dict['Bb'][0][0])
P = int(Dict['P'][0][0])

IELEM = int(Dict['IELEM'][0][0])
NSPV = int(Dict['NSPV'][0][0])

DX = Dict['DX'][0]
ISPV = Dict['ISPV']
VSPV = Dict['VSPV'][0]

In [ ]:
# Figure 2 - Micropolar Hasanyan
BC='SS'
Nu=1/3
E=1
le=E/((1+Nu)*(1-2*Nu))
ue=E/(2*(1+Nu))
g=0
ke=0
ka=0.0001
b=0
Bb=1

num=100
R=[]
X=[]
for i in range(num):
  Hh=1/(3*math.pi)+((4/math.pi - 1/(3*math.pi))/num)*i
  W = IFSDTBuck_MP(le,ke,ue,g,Bb,Hh,P,IELEM,BC,DX)
  R.append(W[0])
  X.append(Hh)

df = pd.DataFrame(R)
df.to_csv('LoadIFSDT.csv',index=False,header=False)
df1 = pd.DataFrame(X)
df1.to_csv('RatioIFSDT.csv',index=False,header=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print(R)

[0.00124, 0.00168, 0.00221, 0.00284, 0.00356, 0.00439, 0.00533, 0.00637, 0.00753, 0.00881, 0.01019, 0.0117, 0.01331, 0.01505, 0.01689, 0.01885, 0.02091, 0.02309, 0.02536, 0.02774, 0.03022, 0.03279, 0.03545, 0.03821, 0.04104, 0.04396, 0.04695, 0.05002, 0.05316, 0.05637, 0.05964, 0.06297, 0.06635, 0.0698, 0.07329, 0.07683, 0.08041, 0.08404, 0.0877, 0.09141, 0.09515, 0.09892, 0.10272, 0.10655, 0.1104, 0.11428, 0.11818, 0.1221, 0.12604, 0.13, 0.13398, 0.13797, 0.14197, 0.14599, 0.15002, 0.15406, 0.1581, 0.16216, 0.16622, 0.17029, 0.17437, 0.17845, 0.18253, 0.18662, 0.19072, 0.19481, 0.19891, 0.20301, 0.20711, 0.21122, 0.21532, 0.21942, 0.22352, 0.22763, 0.23173, 0.23583, 0.23993, 0.24403, 0.24813, 0.25222, 0.25631, 0.26041, 0.26449, 0.26858, 0.27266, 0.27675, 0.28082, 0.2849, 0.28897, 0.29304, 0.29711, 0.30117, 0.30523, 0.30929, 0.31335, 0.3174, 0.32144, 0.32549, 0.32953, 0.33357]


# Calculation of Mesh 1D
Code that calcultae the mesh of the system

In [ ]:
def NLMSH1D(IELEM,IPDF,DX,X0=0):
  """Calculate a 1D mesh with the number of elements and their spaces.

  Args:
    IELEM (int): Number of elements
    IPDF (int): Number of points in an element
    DX (array): Spaces between elements
    X0 (int, optional): Beginning of structure
  
  Returns:
    Array
  """

  NOD = np.zeros((IELEM,IPDF))
  for i in range(IPDF):
    NOD[0,i] = i
  for n in range(1,IELEM):
    for i in range(IPDF):
      NOD[n,i]=NOD[n-1,i]+IPDF-1
  
  # Generate the mesh of a 1D element

  GLX=[X0]
  for i in range(IELEM):
    for j in range(1,IPDF):
      GLX.append(GLX[-1]+DX[i]/(IPDF-1))
  NOD = NOD.astype(int)
  
  return NOD, GLX

# Shape functions
Calculate the Lagrangian Shape functions



In [ ]:
def SHAPE1D(H,NPE,XI):
  """Calculate the Lagrangian Shape functions in one dimension by a given length of 
  element, number of points and point value. Returns an array of values of the 
  shape function and its derivative.

  Args:
    H (float): The length of the element
    NPE (int): Number of points in each element
    XI (array): Values of quadrature points calculated
  
  Returns:
    Arrays
  """

# Calculate the Lagrangian shape functions by a given length H.
  P=NPE-1
  XF=[]
  XDF=[]

  XNODE = np.linspace(-1,1,num=NPE)

  for i in range(NPE):
    r3=np.zeros(NPE)
    s1=np.ones(NPE)
    r1=np.ones(NPE)
    for n in range(NPE):
      if n != i:
        s1 = s1 * (XI-XNODE[n])
        r1 = r1 * (XNODE[i]-XNODE[n])
    XF.append(s1/r1)

    for j in range(NPE):
      s3 = np.ones(NPE)
      for k in range(NPE):
        if j != i:
          if (j != k) & (k != i):
            s3 = s3* (XI-XNODE[k])
            g3 = s3
        else:
          g3 = np.zeros(NPE)
      
      if P == 1:
        r3 = np.ones(NPE)
      else:
        r3 = r3 + g3
    XDF.append(r3 / r1) 

  SF = np.array(XF)
  DSF = np.array(XDF)
  GJ = H*0.5
  GDSF = DSF / GJ

  return SF, GDSF, GJ


# Gauss Points
Calculation of Gauss Points using a User-defined function

In [ ]:
print(np.polynomial.legendre.leggauss(5))

(array([-0.90617985, -0.53846931,  0.        ,  0.53846931,  0.90617985]), array([0.23692689, 0.47862867, 0.56888889, 0.47862867, 0.23692689]))


# Calculation of Local matrices
The main code, we use a combination with Maple.

In [ ]:
def NLCOEFF(IPDF,H,A,I2,le,ue,ke,bet3,NDF=6,AKS=5/6):
  """Calculate the local matrices of lineal and geometrical stiffness.

  Args:
    IPDF (int): Number of points in an element
    H (float): length of the element
    A (float): Area of cross section
    I2 (float): Inercia modulus in direction 2
    le (float): Lame parameter lambda
    ue (float): Micropolar parameter u
    ke (float): Micropolar parameter k
    bet3 (float): Micropolar parameter gamma
    NDF (int, optional): Number of degrees of freedom, IFSDT micropolar = 6
    AKS (float, optional): Shear correction factor, Rectangular = 5/6

  Returns:
    arrays
  """ 

  GAUSPT, GAUSWT = np.polynomial.legendre.leggauss(IPDF)

  SFA, GDSFA, GJ = SHAPE1D(H,IPDF,GAUSPT)
  CNSTA = GJ * GAUSWT

  ELK = np.zeros((IPDF*NDF,IPDF*NDF))
  ELKG = np.zeros((IPDF*NDF,IPDF*NDF))

  ELKk = np.zeros((NDF,NDF,IPDF,IPDF))
  ELKkg = np.zeros((NDF,NDF,IPDF,IPDF))

  A0X = -1

  # Local matrix ELK
  for LI in range(IPDF):
    SF = SFA[:,LI]; GDSF = GDSFA[:,LI]; CNST = CNSTA[LI]
    for II in range(IPDF):
      for JJ in range(IPDF):
        
        # Calculation of Lineal Matrix Kl
        # Coefficient ELK11
        t0 = (A * le * GDSF[JJ] + 0.2e1 * A * ue * GDSF[JJ] + A * ke * GDSF[JJ]) * GDSF[II]
        ELKk[0,0,II,JJ]=ELKk[0,0,II,JJ]+t0*CNST;
        # Coefficient ELK12
        t0 = 0.0e0
        ELKk[0,1,II,JJ]=ELKk[0,1,II,JJ]+t0*CNST;
        # Coefficient ELK13
        t0 = 0.0e0
        ELKk[0,2,II,JJ]=ELKk[0,2,II,JJ]+t0*CNST;
        # Coefficient ELK14
        t0 = A * le * SF[JJ] * GDSF[II]
        ELKk[0,3,II,JJ]=ELKk[0,3,II,JJ]+t0*CNST;
        # Coefficient ELK15
        t0 = 0.0e0
        ELKk[0,4,II,JJ]=ELKk[0,4,II,JJ]+t0*CNST;
        # Coefficient ELK16
        t0 = 0.0e0
        ELKk[0,5,II,JJ]=ELKk[0,5,II,JJ]+t0*CNST;
        # Coefficient ELK21
        t0 = 0.0e0
        ELKk[1,0,II,JJ]=ELKk[1,0,II,JJ]+t0*CNST;
        # Coefficient ELK22
        t0 = (AKS * A * ke * GDSF[JJ] + AKS * A * ue * GDSF[JJ]) * GDSF[II]
        ELKk[1,1,II,JJ]=ELKk[1,1,II,JJ]+t0*CNST;
        # Coefficient ELK23
        t0 = AKS * A * ue * SF[JJ] * GDSF[II]
        ELKk[1,2,II,JJ]=ELKk[1,2,II,JJ]+t0*CNST;
        # Coefficient ELK24
        t0 = 0.0e0
        ELKk[1,3,II,JJ]=ELKk[1,3,II,JJ]+t0*CNST;
        # Coefficient ELK25
        t0 = 0.0e0
        ELKk[1,4,II,JJ]=ELKk[1,4,II,JJ]+t0*CNST;
        # Coefficient ELK26
        t0 = -0.1e1 * AKS * A * ke * SF[JJ] * GDSF[II]
        ELKk[1,5,II,JJ]=ELKk[1,5,II,JJ]+t0*CNST;
        # Coefficient ELK31
        t0 = 0.0e0
        ELKk[2,0,II,JJ]=ELKk[2,0,II,JJ]+t0*CNST;
        # Coefficient ELK32
        t0 = AKS * A * ue * GDSF[JJ] * SF[II]
        ELKk[2,1,II,JJ]=ELKk[2,1,II,JJ]+t0*CNST;
        # Coefficient ELK33
        t0 = (AKS * A * ke * SF[JJ] + AKS * A * ue * SF[JJ]) * SF[II] + (0.2e1 * I2 * ue * GDSF[JJ] + I2 * ke * GDSF[JJ] + I2 * le * GDSF[JJ]) * GDSF[II]
        ELKk[2,2,II,JJ]=ELKk[2,2,II,JJ]+t0*CNST;
        # Coefficient ELK34
        t0 = 0.0e0
        ELKk[2,3,II,JJ]=ELKk[2,3,II,JJ]+t0*CNST;
        # Coefficient ELK35
        t0 = 0.2e1 * I2 * le * SF[JJ] * GDSF[II]
        ELKk[2,4,II,JJ]=ELKk[2,4,II,JJ]+t0*CNST;
        # Coefficient ELK36
        t0 = AKS * A * ke * SF[JJ] * SF[II]
        ELKk[2,5,II,JJ]=ELKk[2,5,II,JJ]+t0*CNST;
        # Coefficient ELK41
        t0 = A * le * GDSF[JJ] * SF[II]
        ELKk[3,0,II,JJ]=ELKk[3,0,II,JJ]+t0*CNST;
        # Coefficient ELK42
        t0 = 0.0e0
        ELKk[3,1,II,JJ]=ELKk[3,1,II,JJ]+t0*CNST;
        # Coefficient ELK43
        t0 = 0.0e0
        ELKk[3,2,II,JJ]=ELKk[3,2,II,JJ]+t0*CNST;
        # Coefficient ELK44
        t0 = (0.2e1 * A * ue * SF[JJ] + A * ke * SF[JJ] + A * le * SF[JJ]) * SF[II] + (AKS * I2 * GDSF[JJ] * ke + AKS * I2 * GDSF[JJ] * ue) * GDSF[II]
        ELKk[3,3,II,JJ]=ELKk[3,3,II,JJ]+t0*CNST;
        # Coefficient ELK45
        t0 = 0.0e0
        ELKk[3,4,II,JJ]=ELKk[3,4,II,JJ]+t0*CNST;
        # Coefficient ELK46
        t0 = 0.0e0
        ELKk[3,5,II,JJ]=ELKk[3,5,II,JJ]+t0*CNST;
        # Coefficient ELK51
        t0 = 0.0e0
        ELKk[4,0,II,JJ]=ELKk[4,0,II,JJ]+t0*CNST;
        # Coefficient ELK52
        t0 = 0.0e0
        ELKk[4,1,II,JJ]=ELKk[4,1,II,JJ]+t0*CNST;
        # Coefficient ELK53
        t0 = 0.2e1 * I2 * le * GDSF[JJ] * SF[II]
        ELKk[4,2,II,JJ]=ELKk[4,2,II,JJ]+t0*CNST;
        # Coefficient ELK54
        t0 = 0.0e0
        ELKk[4,3,II,JJ]=ELKk[4,3,II,JJ]+t0*CNST;
        # Coefficient ELK55
        t0 = (0.8e1 * I2 * ue * SF[JJ] + 0.4e1 * I2 * ke * SF[JJ] + 
              0.4e1 * I2 * le * SF[JJ]) * SF[II]
        ELKk[4,4,II,JJ]=ELKk[4,4,II,JJ]+t0*CNST;
        # Coefficient ELK56
        t0 = 0.0e0
        ELKk[4,5,II,JJ]=ELKk[4,5,II,JJ]+t0*CNST;
        # Coefficient ELK61
        t0 = 0.0e0
        ELKk[5,0,II,JJ]=ELKk[5,0,II,JJ]+t0*CNST;
        # Coefficient ELK62
        t0 = -0.1e1 * AKS * A * ke * GDSF[JJ] * SF[II]
        ELKk[5,1,II,JJ]=ELKk[5,1,II,JJ]+t0*CNST;
        # Coefficient ELK63
        t0 = AKS * A * ke * SF[JJ] * SF[II]
        ELKk[5,2,II,JJ]=ELKk[5,2,II,JJ]+t0*CNST;
        # Coefficient ELK64
        t0 = 0.0e0
        ELKk[5,3,II,JJ]=ELKk[5,3,II,JJ]+t0*CNST;
        # Coefficient ELK65
        t0 = 0.0e0
        ELKk[5,4,II,JJ]=ELKk[5,4,II,JJ]+t0*CNST;
        # Coefficient ELK66
        t0 = 0.2e1 * AKS * A * ke * SF[JJ] * SF[II] + A * bet3 * GDSF[JJ] * GDSF[II]
        ELKk[5,5,II,JJ]=ELKk[5,5,II,JJ]+t0*CNST;


        # Calculation of Geometrical Matrix Kg
        # Coefficient ELKg11
        t0 = 0.0e0
        ELKkg[0,0,II,JJ]=ELKkg[0,0,II,JJ]+t0*CNST;
        # Coefficient ELKg12
        t0 = 0.0e0
        ELKkg[0,1,II,JJ]=ELKkg[0,1,II,JJ]+t0*CNST;
        # Coefficient ELKg13
        t0 = 0.0e0
        ELKkg[0,2,II,JJ]=ELKkg[0,2,II,JJ]+t0*CNST;
        # Coefficient ELKg14
        t0 = 0.0e0
        ELKkg[0,3,II,JJ]=ELKkg[0,3,II,JJ]+t0*CNST;
        # Coefficient ELKg15
        t0 = 0.0e0
        ELKkg[0,4,II,JJ]=ELKkg[0,4,II,JJ]+t0*CNST;
        # Coefficient ELKg16
        t0 = 0.0e0
        ELKkg[0,5,II,JJ]=ELKkg[0,5,II,JJ]+t0*CNST;
        # Coefficient ELKg21
        t0 = 0.0e0
        ELKkg[1,0,II,JJ]=ELKkg[1,0,II,JJ]+t0*CNST;
        # Coefficient ELKg22
        t0 = - A0X * GDSF[JJ] * GDSF[II]
        ELKkg[1,1,II,JJ]=ELKkg[1,1,II,JJ]+t0*CNST;
        # Coefficient ELKg23
        t0 = 0.0e0
        ELKkg[1,2,II,JJ]=ELKkg[1,2,II,JJ]+t0*CNST;
        # Coefficient ELKg24
        t0 = 0.0e0
        ELKkg[1,3,II,JJ]=ELKkg[1,3,II,JJ]+t0*CNST;
        # Coefficient ELKg25
        t0 = 0.0e0
        ELKkg[1,4,II,JJ]=ELKkg[1,4,II,JJ]+t0*CNST;
        # Coefficient ELKg26
        t0 = 0.0e0
        ELKkg[1,5,II,JJ]=ELKkg[1,5,II,JJ]+t0*CNST;
        # Coefficient ELKg31
        t0 = 0.0e0
        ELKkg[2,0,II,JJ]=ELKkg[2,0,II,JJ]+t0*CNST;
        # Coefficient ELKg32
        t0 = 0.0e0
        ELKkg[2,1,II,JJ]=ELKkg[2,1,II,JJ]+t0*CNST;
        # Coefficient ELKg33
        t0 = 0.0e0
        ELKkg[2,2,II,JJ]=ELKkg[2,2,II,JJ]+t0*CNST;
        # Coefficient ELKg34
        t0 = 0.0e0
        ELKkg[2,3,II,JJ]=ELKkg[2,3,II,JJ]+t0*CNST;
        # Coefficient ELKg35
        t0 = 0.0e0
        ELKkg[2,4,II,JJ]=ELKkg[2,4,II,JJ]+t0*CNST;
        # Coefficient ELKg36
        t0 = 0.0e0
        ELKkg[2,5,II,JJ]=ELKkg[2,5,II,JJ]+t0*CNST;
        # Coefficient ELKg41
        t0 = 0.0e0
        ELKkg[3,0,II,JJ]=ELKkg[3,0,II,JJ]+t0*CNST;
        # Coefficient ELKg42
        t0 = 0.0e0
        ELKkg[3,1,II,JJ]=ELKkg[3,1,II,JJ]+t0*CNST;
        # Coefficient ELKg43
        t0 = 0.0e0
        ELKkg[3,2,II,JJ]=ELKkg[3,2,II,JJ]+t0*CNST;
        # Coefficient ELKg44
        t0 = - 0* A0X / (2 * ue + ke + le) * le * SF[JJ] * SF[II]
        ELKkg[3,3,II,JJ]=ELKkg[3,3,II,JJ]+t0*CNST;
        # Coefficient ELKg45
        t0 = 0.0e0
        ELKkg[3,4,II,JJ]=ELKkg[3,4,II,JJ]+t0*CNST;
        # Coefficient ELKg46
        t0 = 0.0e0
        ELKkg[3,5,II,JJ]=ELKkg[3,5,II,JJ]+t0*CNST;
        # Coefficient ELKg51
        t0 = 0.0e0
        ELKkg[4,0,II,JJ]=ELKkg[4,0,II,JJ]+t0*CNST;
        # Coefficient ELKg52
        t0 = 0.0e0
        ELKkg[4,1,II,JJ]=ELKkg[4,1,II,JJ]+t0*CNST;
        # Coefficient ELKg53
        t0 = 0.0e0
        ELKkg[4,2,II,JJ]=ELKkg[4,2,II,JJ]+t0*CNST;
        # Coefficient ELKg54
        t0 = 0.0e0
        ELKkg[4,3,II,JJ]=ELKkg[4,3,II,JJ]+t0*CNST;
        # Coefficient ELKg55
        t0 = 0.0e0
        ELKkg[4,4,II,JJ]=ELKkg[4,4,II,JJ]+t0*CNST;
        # Coefficient ELKg56
        t0 = 0.0e0
        ELKkg[4,5,II,JJ]=ELKkg[4,5,II,JJ]+t0*CNST;
        # Coefficient ELKg61
        t0 = 0.0e0
        ELKkg[5,0,II,JJ]=ELKkg[5,0,II,JJ]+t0*CNST;
        # Coefficient ELKg62
        t0 = 0.0e0
        ELKkg[5,1,II,JJ]=ELKkg[5,1,II,JJ]+t0*CNST;
        # Coefficient ELKg63
        t0 = 0.0e0
        ELKkg[5,2,II,JJ]=ELKkg[5,2,II,JJ]+t0*CNST;
        # Coefficient ELKg64
        t0 = 0.0e0
        ELKkg[5,3,II,JJ]=ELKkg[5,3,II,JJ]+t0*CNST;
        # Coefficient ELKg65
        t0 = 0.0e0
        ELKkg[5,4,II,JJ]=ELKkg[5,4,II,JJ]+t0*CNST;
        # Coefficient ELKg66
        t0 = 0.0e0
        ELKkg[5,5,II,JJ]=ELKkg[5,5,II,JJ]+t0*CNST;

  # Matrix Rearrenge
  II = 0
  for I in range(IPDF):
    JJ= 0
    for J in range(IPDF):
      for i in range(NDF):
        for j in range(NDF):
          ELK[II+i,JJ+j] = ELKk[i,j,I,J]
          ELKG[II+i,JJ+j] = ELKkg[i,j,I,J]
      JJ = (J+1)*NDF
    II = (I+1)*NDF

  return ELK, ELKG

# Assembly of global matrices
Assemble the matrices from local to global forms

In [ ]:
def NLASSEMBLE(NI,IPDF,GLK,GLKG,ELK,ELKG,NDF=6):

  # Calculation of the index of assembly
  index=[]
  for i in range(IPDF):
    start = NI[i] * NDF
    for j in range(NDF):
      index.append(start+j)

  # Assembly of element stifness matrix into the system matrix
  for i in range(len(index)):
    ii = index[i]
    for j in range(len(index)):
      jj = index[j]
      GLK[ii,jj] = GLK[ii,jj] + ELK[i,j]
      GLKG[ii,jj] = GLKG[ii,jj] + ELKG[i,j]

  return GLK, GLKG


# Calculation of stifness and geometrical matrices
General code where the global stiffness matrix is calculated


In [ ]:
def NLSTIFF(IELEM,NEQ,NOD,GLX,GLK,GLKG,IPDF,A,I2,le,ue,ke,bet3,NDF=6,AKS=5/6):
  """Calculate the Lineal and Geometrical global matrices.

  Args:
    IELEM (int): Number of elements
    NEQ (int): Number of equations
    NOD (array): Array of nodes connectivity
    GLX (array): Global position of nodes
    GLK (array): Global lineal matrix
    GLKG (array): Global geometrical matrix
    IPDF (int): Number of nodes per element
    A (float): Area of cross section
    I2 (float): Second moment of inertia in direction 2
    le (float): Lame parameter lambda
    ue (float): Eringen micropolar parameter u
    ke (float): Eringen micropolar parameter k
    bet3 (float): Micropolar parameter gamma
    NDF (int,optional): Number of degrees of freedom, IFSDT micropolar = 6
    AKS (float, optional): Shear correction factor, Rectangular = 5/6

  Returns:
    Array
  """

  for NE in range(IELEM):
    ELX = []
    for i in range(IPDF):
      NI = int(NOD[NE,i])
      ELX.append(GLX[NI])

    # Initiate local matrices
    H = ELX[-1] - ELX[0]
    ELK, ELKG = NLCOEFF(IPDF,H,A,I2,le,ue,ke,bet3,NDF,AKS)

    # Assembly of matrices
    NI= NOD[NE,:]
    GLK, GLKG = NLASSEMBLE(NI,IPDF,GLK,GLKG,ELK,ELKG)

  return GLK, GLKG

# Boundary Conditions Code
Code for establishing the boundary conditions in the global matrices

In [ ]:
def NLSYMBNDRY(NSPV,ISPV,VSPV,GLK,GLKG,NDF=6):
  """Establish the boundary condition in the global matrices

  Args:
    NSPV (int): Number of primary boundary conditions
    ISPV (array): Position of primary boundary conditions
    VSPV (array): Value of primary boundary conditions
    GLK (array): Global stiffness matrix of the system
    GLKG (array): Global geometrical matrix of the system
    NDF (int,optional): Number of degrees of freedom

  Returns:
    Arrays
  """

  # Primary degrees of freedom
  if NSPV != 0:
    for NB in range(NSPV):
      FDF = (ISPV[NB,0]-1)*NDF+ISPV[NB,1]-1
      GLK[:,FDF] = 0
      GLK[FDF,:] = 0
      GLK[FDF,FDF] = 1
      GLKG[:,FDF] = 0
      GLKG[FDF,:] = 0
      GLKG[FDF,FDF] = 1

  return GLK, GLKG

# Plot Mode Shapes


In [ ]:
def PlotModeShapes(Val,Vec,GLX,NDF=6,NMODES=4):
  """Plot the buckling modes of a beam with their eignevalues and eigenvectors.

  Args:
    Val(array): Eigenvector of the beam
    Vec(array): Eigenvalues of the beam
    GLX(array): Global position of nodes
    NDF(int, optional): Number of degrees of freedom
    NMODES(int, optional): Number of modes plotted

  Returns:
    Plot
    Array
  """


  Vy=Vec[1::NDF,:]

  # Normalize the eignevector
  VN=[Vy[:,i]/max(Vy[:,i],key=abs) for i in range(len(Vy[0]))]
  VN = np.array([*zip(*VN)])

  # Sort the vector matrix and modes to minimum and maximum
  indx = Val.argsort(axis=0)
  Valnew = Val[indx]
  VN = VN[ : , indx ]

  # Extracting all terms Nan
  nbc = 0
  for i in range(len(Valnew)):
    if Valnew[i] < 0:
      nbc+= 1
    else: 
      break

  # Plotting mode shapes
  L = max(GLX)
  DataX = np.transpose(VN[:,0:NMODES])
  DataY = [GLX] * NMODES

  XLIM=[[-1,1]]*NMODES
  YLIM=[[0,1]]*NMODES

  #fig, axs = plt.subplots(1, NMODES)
  #fig.suptitle('Mode Shapes in Micropolar Beam')
  #for j, (x, y, xlim, ylim) in enumerate(zip(DataX,DataY,XLIM,YLIM)):
  #  axs[j].plot(x , y, marker='o', color='b')
  #  axs[j].set_title(Valnew[j])
  #  axs[j].set_xlim(xlim)
  #  axs[j].set_ylim(ylim)
  
  return Valnew
  

  




# Main Code
This is the main code: MPBEAM 1.0


In [ ]:
def IFSDTBuck_MP(le,ke,ue,bet3,Bb,Hh,P,IELEM,BC,DX,NDF=6):
  """ Micropolar IFSDT 1d beam Code. Calculate the buckling modes and critical
  loads of any beam with a rectangular cross section using Finite Element Method.
  Returns plots and the eigenvector and eigenvalues calculated.

  Args:
    le (float): Lambda Lame coeficcient lambda
    ke (float): Micropolar parameter k
    ue (float): Eringen Lame coeficient u
    bet3 (float): Micropolar parameter gamma
    Bb (float): Base of the cross section
    Hh (float): Height og the beam
    P (int): Degree of polynomial
    IELEM (int): Number of elements
    BC (str): Boundary condition
    DX (array): Space between elements
    NDF (int,optional): Number of degrees of freedom

  Returns:
    Arrays
    Plots (optional)
  """
  # Material properties
  Em=(2*ue+ke)*(3*le+2*ue+ke)/(2*le+2*ue+ke)
  Num=le/(2*ue+2*le+ke)
  ke = 2*ka
  ue = ue-0.5*ke

  # Cross sectional properties
  A = Bb*Hh; I2=Bb*(Hh**3) / 12;
  AKS = 5/6

  # Geometry
  IPDF = P+1
  NNM = IELEM*P+1
  NN = IPDF*NDF
  NEQ = NNM*NDF
  NOD, GLX = NLMSH1D(IELEM,IPDF,DX)

  # Boundary Conditions 
  Boundary = {'CF':{'NSPV':5,'ISPV':[[1,1],[1,2],[1,3],[1,4],[1,5]],
                    'VSPV':np.zeros(5)},
            'SS':{'NSPV':3,'ISPV':[[1,1],[1,2],[NNM,2]],'VSPV':np.zeros(3)},
            'CS':{'NSPV':6,'ISPV':[[1,1],[1,2],[1,3],[1,4],[1,5],[NNM,2]],
                  'VSPV':np.zeros(6)},
            'CC':{'NSPV':10,'ISPV':[[1,1],[1,2],[1,3],[1,4],[1,5],
                                    [NNM,1],[NNM,2],[NNM,3],[NNM,4],[NNM,5]],
                  'VSPV':np.zeros(10)}}
  NSPV=Boundary[BC]['NSPV']
  ISPV=np.array(Boundary[BC]['ISPV'])
  VSPV=np.array(Boundary[BC]['VSPV'])

  # Calculation of the stifness matrices
  GLK = np.zeros((NEQ,NEQ))
  GLKG = np.zeros((NEQ,NEQ))
  GLK, GLKG = NLSTIFF(IELEM,NEQ,NOD,GLX,GLK,GLKG,IPDF,A,I2,le,ue,ke,bet3,NDF)
  # Imposition of boundary condition
  GLK, GLKG = NLSYMBNDRY(NSPV,ISPV,VSPV,GLK,GLKG)
  np.set_printoptions(edgeitems=68)
  # Calculation of eigenvalues and eignevectors
  W, V = eig(GLK,GLKG)
  W = np.real(W)
  W = np.around(W,decimals=5)
  W = PlotModeShapes(W,V,GLX)

  return W